# Deep Learning Inference using Pretrained AI Models

In [ ]:
import os
import pathlib
import scipy.misc
import tarfile
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from io import BytesIO
from six.moves import urllib
from six.moves.urllib.request import urlopen
from matplotlib import gridspec
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'    # Suppress TensorFlow logging (1)
tf.get_logger().setLevel('ERROR')

# Image Classification

## Step 1: Load the Pretrained Models

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
model = ResNet50(weights='imagenet')

## Step 2: Load and preprocess an image from the Internet

In [ ]:
url = 'https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fspecials-images.forbesimg.com%2Fimageserve%2F5db4c7b464b49a0007e9dfac%2FPhoto-of-Maltese-dog%2F960x0.jpg%3Ffit%3Dscale'
url = 'https://i.guim.co.uk/img/media/684c9d087dab923db1ce4057903f03293b07deac/205_132_1915_1150/master/1915.jpg?width=1020&quality=45&auto=format&fit=max&dpr=2&s=a634e9501725b30f045481c39ac2dd1d'
f = urllib.request.urlopen(url)
jpeg_str = f.read()
original_im = Image.open(BytesIO(jpeg_str))
original_im = original_im.resize((224,224))
original_im = np.array(original_im).reshape(224,224,3)
plt.imshow(original_im)

## Step 3: Carry out Inference on the pretrained model using model.predict()

In [ ]:
predictions = model.predict(np.expand_dims(original_im, axis=0))
predicted_class = decode_predictions(predictions, top=1)

plt.imshow(original_im)
plt.axis('off')
_ = plt.title("Prediction: " + predicted_class[0][0][1])

# Object Detection

In [ ]:
%%bash
git clone https://github.com/tensorflow/models.git

cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

## Step 1: Load the Pretrained Model

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Function to Download and extract model
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    return str(model_dir)

# Function to Download labels file
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)


MODEL_DATE = '20200711'
MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

**Load the Pretrained Model**

In [ ]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

## Step 2: Load the image

In [ ]:
url = 'https://i5.walmartimages.com/asr/12516853-fa35-46ff-9678-5829bc037e5c.34f02985dade5d86beac256fb2e6c1dd.jpeg?odnHeight=612&odnWidth=612&odnBg=FFFFFF'
f = urllib.request.urlopen(url)
jpeg_str = f.read()
original_im = Image.open(BytesIO(jpeg_str))
original_im = np.array(original_im)
plt.imshow(original_im)

input_tensor = tf.convert_to_tensor(original_im)
input_tensor = input_tensor[tf.newaxis, ...]

In [ ]:
## Step 3: Carry out inference using the Pretrained model

In [ ]:
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = original_im.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False)

plt.figure(figsize=(18, 16), dpi=80)

plt.imshow(image_np_with_detections)
print('Done')

# Natural Language Processing Examples
# Sentiment Analysis Engine

In [ ]:
!pip install -U pysentimiento

In [ ]:
from pysentimiento import EmotionAnalyzer
emotion_analyzer = EmotionAnalyzer(lang="en")

In [ ]:
emotion_analyzer.predict("This is so terrible...")

In [ ]:
emotion_analyzer.predict("It is ridiculous, I am over this.")